In [1]:
#librerias
import consultas as cst
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt   

In [2]:
#mejor visualizacion de las tablas
pd.options.display.max_columns = False

In [ ]:
#QUERY QUE TRAE LOS USUARIOS DEL ULTIMO ENDING QUE CORRERA EL MODELO , CAMBIAR EL PROCESOCODIGO  PARA ENTRENAR
query=f'''
SELECT A.SUBSCRIBER_ID,
       A.MOVIMIENTO_NOMBRE 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=TO_NUMBER(TO_CHAR(SYSTIMESTAMP,'YYYYMM'))
AND A.SERVICIO='Prepaid'
AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
AND A.ESTADO='Active'
AND A.SUBSCRIBER_ID IN (
            SELECT SUBSCRIBER_ID FROM ( 
                    SELECT B.*,
                    ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                    FROM AGG_DL_PJ_QUALITY B )
            WHERE RN=1
            AND   PREDICCION=0
)
'''

In [ ]:
#QUERY QUE NOS DA LA ULTIMA RECARGA QUE HA TENIDO CADA SUBSCRIBER DEL FILTRO 
query=f'''
SELECT B.SUBSCRIBER_ID,
       B.TIEMPO_RECARGA_DK AS TIEMPO_ULTIMA_RECARGA,
       B.VALOR_CARGA AS VALOR_ULTIMA_RECARGA,
       B.FECHA_RECARGA AS ULTIMA_RECARGA
FROM (
SELECT B.*,
       ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_RECARGA DESC) AS RN  
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
WHERE B.SUBSCRIBER_ID IN (
        SELECT A.SUBSCRIBER_ID 
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
        WHERE A.PERIODO_PROCESO_CODIGO=TO_NUMBER(TO_CHAR(SYSTIMESTAMP,'YYYYMM'))
        AND A.SERVICIO='Prepaid'
        AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
        AND A.ESTADO='Active'
        AND A.SUBSCRIBER_ID IN (
                SELECT SUBSCRIBER_ID FROM ( 
                        SELECT B.*,
                        ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                        FROM AGG_DL_PJ_QUALITY B )
                WHERE RN=1
                AND   PREDICCION=0
        )
) )B
WHERE B.RN=1


'''

In [ ]:
#QUERY QUE UNE LAS 2 TABLAS ANTERIORES Y LO DA PARA LA ACTUAL 

query=f'''
WITH BASE AS 
(SELECT /*PARALLEL(8)*/
        A.SUBSCRIBER_ID,
        A.MOVIMIENTO_NOMBRE 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=TO_NUMBER(TO_CHAR(SYSTIMESTAMP,'YYYYMM'))
AND A.SERVICIO='Prepaid'
AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
AND A.ESTADO='Active'
AND A.SUBSCRIBER_ID IN (
            SELECT SUBSCRIBER_ID FROM ( 
                    SELECT B.*,
                    ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                    FROM AGG_DL_PJ_QUALITY B )
            WHERE RN=1
            AND   PREDICCION=0
)),
RECARGAS AS (
SELECT /*PARALLEL(8)*/
       B.SUBSCRIBER_ID,
       B.TIEMPO_RECARGA_DK AS TIEMPO_ULTIMA_RECARGA,
       B.VALOR_CARGA AS VALOR_ULTIMA_RECARGA,
       B.FECHA_RECARGA AS ULTIMA_RECARGA
FROM (
SELECT /*PARALLEL(8)*/
       B.*,
       ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_RECARGA DESC) AS RN  
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
WHERE B.SUBSCRIBER_ID IN (
        SELECT A.SUBSCRIBER_ID 
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
        WHERE A.PERIODO_PROCESO_CODIGO=TO_NUMBER(TO_CHAR(SYSTIMESTAMP,'YYYYMM'))
        AND A.SERVICIO='Prepaid'
        AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
        AND A.ESTADO='Active'
        AND A.SUBSCRIBER_ID IN (
                SELECT SUBSCRIBER_ID FROM ( 
                        SELECT B.*,
                        ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                        FROM AGG_DL_PJ_QUALITY B )
                WHERE RN=1
                AND   PREDICCION=0
        )
) )B
WHERE B.RN=1
)

SELECT  /*PARALLEL(8)*/
        A.*,
        B.*,
        CASE 
            WHEN B.ULTIMA_RECARGA >= SYSDATE-5 THEN TO_NUMBER(TO_CHAR(B.ULTIMA_RECARGA,'YYYYMMDD'))
            ELSE TO_NUMBER(TO_CHAR(B.ULTIMA_RECARGA+ROUND((SYSDATE-B.ULTIMA_RECARGA)/5,0)*5,'YYYYMMDD')) 
        END AS INICIO_CICLO,
        CASE 
            WHEN B.ULTIMA_RECARGA = SYSDATE-5  THEN B.ULTIMA_RECARGA
        END AS FECHA_PRED,
        ROUND((SYSDATE-B.ULTIMA_RECARGA)/5,0) AS CICLOS 
        
FROM BASE A 
JOIN RECARGAS B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID 
'''

In [ ]:
#COGEMOS EL QUERY ANTERIOR Y SEGMENTAMOS PARA ENTRENAR CON FECHA 20221120 NOVIEMBRE
#QUERY QUE UNE LAS 2 TABLAS ANTERIORES Y LO DA PARA LA ACTUAL 
#LA IDEA DESPUES ES PEGARLE MAS DIAS CON LA MISMA LOGICA QUE SE ESTA MANEJANDO PARA TENER MAS BASE DE ENTRENAMIENTO 
#esta base contiene variables de ciclos , valor ultima_recarga y si es gross o permanece 

query=f'''
WITH BASE AS 
(SELECT /*PARALLEL(8)*/
        A.SUBSCRIBER_ID,
        A.MOVIMIENTO_NOMBRE 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
WHERE A.PERIODO_PROCESO_CODIGO=202211
AND A.SERVICIO='Prepaid'
AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
AND A.ESTADO='Active'
AND A.SUBSCRIBER_ID IN (
            SELECT SUBSCRIBER_ID FROM ( 
                    SELECT B.*,
                    ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                    FROM AGG_DL_PJ_QUALITY B )
            WHERE RN=1
            AND   PREDICCION=0
)),
RECARGAS AS (
SELECT /*PARALLEL(8)*/
       B.SUBSCRIBER_ID,
       B.TIEMPO_RECARGA_DK AS TIEMPO_ULTIMA_RECARGA,
       B.VALOR_CARGA AS VALOR_ULTIMA_RECARGA,
       B.FECHA_RECARGA AS ULTIMA_RECARGA
FROM (
SELECT /*PARALLEL(8)*/
       B.*,
       ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_RECARGA DESC) AS RN  
FROM DWH_BODEGA_WOM.FCT_RECARGAS B
WHERE   B.TIEMPO_RECARGA_DK<=20221120
        AND B.SUBSCRIBER_ID IN (
        SELECT A.SUBSCRIBER_ID 
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A 
        WHERE A.PERIODO_PROCESO_CODIGO=202211
        AND A.SERVICIO='Prepaid'
        AND A.MOVIMIENTO_NOMBRE IN ('ACTIVACION','PERMANECE')
        AND A.ESTADO='Active'
        AND A.SUBSCRIBER_ID IN (
                SELECT SUBSCRIBER_ID FROM ( 
                        SELECT B.*,
                        ROW_NUMBER() OVER (PARTITION BY SUBSCRIBER_ID ORDER BY FECHA_PRED DESC) AS RN 
                        FROM AGG_DL_PJ_QUALITY B )
                WHERE RN=1
                AND   PREDICCION=0
        )
) )B
WHERE B.RN=1
)
SELECT * FROM  (
SELECT  /*PARALLEL(8)*/
        A.MOVIMIENTO_NOMBRE,
        B.*,
        CASE
            WHEN A.MOVIMIENTO_NOMBRE='ACTIVACION' THEN 1 
            ELSE 0 
        END AS GROSS_PERMA,
        CASE 
            WHEN B.TIEMPO_ULTIMA_RECARGA >= 20221120-5 THEN B.TIEMPO_ULTIMA_RECARGA
            ELSE TO_NUMBER(TO_CHAR(B.ULTIMA_RECARGA+ROUND((TO_DATE(TO_CHAR(20221120), 'YYYY/MM/DD')-B.ULTIMA_RECARGA)/5,0)*5 -5,'YYYYMMDD')) 
        END AS INICIO_CICLO,
        20221120 AS FECHA_PRED,
        ROUND((TO_DATE(TO_CHAR(20221120), 'YYYY/MM/DD')-B.ULTIMA_RECARGA)/5,0)-1 AS CICLOS 
FROM BASE A 
JOIN RECARGAS B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID 
)G WHERE INICIO_CICLO=20221120-5


'''
base_entreno=cst.consulta(query)

In [ ]:
#variables trafico cantidad llamadas, duracion y dias con llamadas
query=f'''
SELECT  SUBSCRIBER_ID,
        SUM(CANT_CALLS) AS CANTIDAD_LLAMADAS,
        SUM(DURACION_CALLS) AS DURACION_LLAMADAS,
        COUNT(DISTINCT(PERIODO_PROCESO_CODIGO)) AS DIAS_LLAMADAS
FROM AGG_DL_VOICE
WHERE PERIODO_PROCESO_CODIGO BETWEEN 20221120-5 AND 20221120-1
      AND SERVICIO='Prepaid'
      AND SENTIDO='SALIENTE'
GROUP BY SUBSCRIBER_ID
'''
trafico_voz=cst.consulta(query)

In [ ]:
#variables de trafico de datos 
query=f'''
SELECT SUBSCRIBER_ID,
       ROUND(SUM(BYTES_TOTALES)/1024/1024,0) AS MB_CONSUMIDAS ,
       COUNT(DISTINCT(PERIODO_PROCESO_CODIGO)) AS DIAS_NAV 
FROM AGG_DL_DATA
WHERE PERIODO_PROCESO_CODIGO BETWEEN 20221120-5 AND 20221120-1
      AND SERVICIO='Prepaid'
GROUP BY SUBSCRIBER_ID
'''
trafico_datos=cst.consulta(query)

In [ ]:
#agregar la variable binaria de paquete y los dias restantes de este pegandole el nombre 
query=f'''
SELECT C.*,
       ROUND(TO_DATE(TO_CHAR(20221120), 'YYYY/MM/DD')-C.FECHA_VENCE,0) AS DIAS_VENCIDO
FROM (
SELECT A.SUBSCRIBER_ID,
       A.FECHA_RECARGA,
       A.PAQUETE_NOMBRE,
       COALESCE(B.DURACION,3),
       A.FECHA_RECARGA+COALESCE(B.DURACION,3) AS FECHA_VENCE, 
       ROW_NUMBER() OVER(PARTITION BY A.SUBSCRIBER_ID ORDER BY A.FECHA_RECARGA DESC ) AS RN
FROM DWH_BODEGA_WOM.FCT_PAQUETES A
JOIN PAQUETES B ON A.PAQUETE_NOMBRE=B.PAQUETE_NOMBRE
WHERE TIEMPO_PAQUETE_DK<=20221120-5
      AND SERVICIO='Prepaid'
    ) C WHERE RN=1
'''
paquetes_dur=cst.consulta(query)

In [ ]:
#variable "y" para medir los resultados es mirar si recargo entre el 20 y 25 nov
query=f'''
SELECT SUBSCRIBER_ID,
       CASE
          WHEN COUNT(*)>=1 THEN 1 ELSE 0 
       END AS RECARGO
FROM DWH_BODEGA_WOM.FCT_RECARGAS
WHERE TIEMPO_RECARGA_DK BETWEEN 20221120+1 AND 20221120+5 
GROUP BY SUBSCRIBER_ID
'''
var_y=cst.consulta(query)

In [ ]:
trafico_datos['SUBSCRIBER_ID'] = trafico_datos['SUBSCRIBER_ID'].astype('int64')
trafico_voz['SUBSCRIBER_ID'] = trafico_voz['SUBSCRIBER_ID'].astype('int64')


In [ ]:
df_insumos=base_entreno.merge(trafico_voz,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(trafico_datos,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(paquetes_dur,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')
df_insumos=df_insumos.merge(var_y,left_on='SUBSCRIBER_ID',right_on='SUBSCRIBER_ID',how='left')

In [ ]:
df_insumos=df_insumos.fillna(0)

In [ ]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
df_insumos.columns

In [ ]:
X_prueba=df_insumos[['VALOR_ULTIMA_RECARGA','GROSS_PERMA','CICLOS', 'CANTIDAD_LLAMADAS', 'DURACION_LLAMADAS',
       'DIAS_LLAMADAS', 'MB_CONSUMIDAS', 'DIAS_NAV','DIAS_VENCIDO']]

In [ ]:
y_port = df_insumos['RECARGO']

In [ ]:
X_port=X_prueba
X_train, X_test, y_train, y_test = train_test_split(X_port, y_port, train_size=0.7, random_state=13)

In [ ]:
random_forest = RandomForestClassifier(random_state=12)

In [ ]:
smote = SMOTE(random_state=13)
X_smote, y_smote = smote.fit_resample(X_train, y_train.ravel())

In [ ]:
random_forest.fit(X_smote, y_smote)

In [ ]:
y_pred = random_forest.predict(X_test)

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred)/len(y_test)*100, annot=True)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
feature_importance = random_forest.feature_importances_
feature_names=X_port.columns
dic_feature_importance = {
                        'feature_importance':feature_importance,
                        'feature_names':feature_names }
feature_importance_DF= pd.DataFrame(dic_feature_importance).sort_values(by='feature_importance',ascending=False)
feature_importance_DF.head(25)